<a href="https://colab.research.google.com/github/gelabgaboo/ACP-peptide/blob/main/Final_project_code_Gabriel_Gelabert_(RUN_THIS_ONE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* This file contains the links to download the fine tuned models as well as running them

In [ ]:
# THIS FILE IS THE ONE TO RUN

In [ ]:
!pip install transformers
import os
import pandas as pd
import torch
import re
import numpy as np

from transformers import  BertForSequenceClassification,  AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.9 MB/s eta 0:00:00


In [ ]:
# This is all written by me, Gabriel Gelabert


# import my version of the model from my hugging face repository

from transformers import AutoTokenizer, BertForSequenceClassification
model_name = "gelabgaboo/results"
tokenizer = AutoTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False)
model = BertForSequenceClassification.from_pretrained("gelabgaboo/results")

* This section of code was written by the origianl author
* This uses the imported model and asks the user to input text. Running this section will then determine if the inputed text is an anitmicrobial peptide (AMP) sequence.
* LFIFFF is an AMP found from https://dbaasp.org/search and is not apart of the original training or validation dataset. You can input this to see how the prediction works.

In [ ]:
# This is from the original author
# They use this code to allow a user to imput a peptide sequence
# The model then decides if the inputed sequence is in fact a antimicrobial peptide or not




# predict AMP/non-AMP for a single example (default ex. is from external test data: DRAMP00126)

#@markdown **Input peptide sequence (upper case only)**
input_seq = 'lfifff' #@param {type:"string"}
input_seq_spaced = ' '.join([ input_seq[i:i+1] for i in range(0, len(input_seq), 1) ])
input_seq_spaced = re.sub(r'[UZOB]', 'X', input_seq_spaced)
input_seq_tok = tokenizer(input_seq_spaced, return_tensors = 'pt')

output = model(**input_seq_tok)
logits = output[0]

# extract AMP class probability and make binary prediction
y_prob = torch.sigmoid(logits)[:,1].detach().numpy()
y_pred = y_prob > 0.5
if y_pred == True:
  input_class = 'AMP'
else:
  input_class = 'non-AMP'

print('Input peptide sequence: ' + input_seq)
print('Class prediction: ' + input_class)
print('Probability of being AMP: ', y_prob)

Input peptide sequence: lfifff
Class prediction: non-AMP
Probability of being AMP:  [0.49145845]


* This section will take a random set of sequences from the imported dataset and pass it through the prediction to see if they are antimicrobial peptides or not.
* This dataset is made up of 1,000 different peptides and was found on the huggingfaces public datasets page.
* a random 5% selection of this new dataset will passed through the model and will output its prediction of if the peptide is an Antimicrobial peptide.

In [ ]:
# Loading the new dataset

base_url = "https://huggingface.co/datasets/gelabgaboo/Anticancer-peptide/raw/main/ACPs_Breast_cancer.csv"
df = pd.read_csv(base_url, index_col = 0)

In [ ]:
# formating and selecting 5% of the data
df = pd.DataFrame(data = df)
df = df.sample(frac=.05, random_state = 42)
df = np.array(df)
df = df[:,0]
print(df)

['AADIFSKFKKDMEVKFA' 'PLTKGILGFVFTLT' 'QIPRIAREKIRGLTEYF' 'RVNAAIPNIIV'
 'SKDALMLEAFEQLLGKRRELLGE' 'SFDRVMNAIEEN' 'SVADHSYRVAFITLLLAEELKKK'
 'VMPSLLQAFDLTA' 'KEEGYFYFRSRLSQ' 'MRKWFHNVLSSGQLLADKWPAWDYNRK'
 'AFFTIAAVARVLD' 'DLQLNANAERINVYY' 'FAKLWAKLAKKL' 'HPFYQALGQLARLTLAQWQAQL'
 'YKKFRQLIQVN' 'FALKALKKLKKALKKAL' 'GEGWAKSILFN' 'AIEKEILIEAISS'
 'GLFDIVKKVVGAFGSL' 'RVIEGIKNVIPKLLTWWP' 'GAPLARMEVTLALESLFGRF'
 'LSTVSNFFMNA' 'EQEILRQVKKAYDRAARLG' 'AFRHSVKEELNYIRRRLERFPNRL'
 'FAKKLAKKLL' 'NLSTLAALVAAAG' 'KVRGLIEIISN' 'HFGSLLLLAAWLAEN'
 'NQDELEGYNLFKGSG' 'FAKKLKKLAKKL' 'SPEHRQELIER' 'NQPFALEMIKSRQKKD'
 'PSDQQNEMVRE' 'DAAALFRLHKFLTKW' 'FALALKALKKL' 'KWKFKKIPKFLHLAKKF'
 'DPEKVLFLAGNLLARLTE' 'KWKLFKKALKKLKKALKKAL' 'SEEVLAMVSRKM'
 'RGFRGWWLASTILLLVAEK' 'FAKLLAKALKKFL' 'EALVHKLTERM' 'DIHAAIEMLLG'
 'IPAAKINWDRDIEYMAGILEN' 'FLGMIPGLIGGLISAFK' 'DRQLRSSIERVISIFQS'
 'MTDQQAEARAF']


* Dont type anything into the input sequence here! It will mess up the for loop.
* this section loops the selected peptides through the model and outputs the models prediction with the probability of it being an AMP

In [ ]:


for x in df:
  input_seq = x #@param {type:"string"}
  input_seq_spaced = ' '.join([ input_seq[i:i+1] for i in range(0, len(input_seq), 1) ])
  input_seq_spaced = re.sub(r'[UZOB]', 'X', input_seq_spaced)
  input_seq_tok = tokenizer(input_seq_spaced, return_tensors = 'pt')

  output = model(**input_seq_tok)
  #ouput = model(x)
  logits = output[0]

  # extract AMP class probability and make binary prediction
  y_prob = torch.sigmoid(logits)[:,1].detach().numpy()
  y_pred = y_prob > 0.5
  if y_pred == True:
    input_class = 'AMP'
  else:
    input_class = 'non-AMP'

  print('Input peptide sequence: ' + input_seq)
  print('Class prediction: ' + input_class)
  print('Probability of being AMP: ', y_prob)

Input peptide sequence: AADIFSKFKKDMEVKFA
Class prediction: non-AMP
Probability of being AMP:  [0.47397003]
Input peptide sequence: PLTKGILGFVFTLT
Class prediction: AMP
Probability of being AMP:  [0.5261148]
Input peptide sequence: QIPRIAREKIRGLTEYF
Class prediction: non-AMP
Probability of being AMP:  [0.47124013]
Input peptide sequence: RVNAAIPNIIV
Class prediction: non-AMP
Probability of being AMP:  [0.4760462]
Input peptide sequence: SKDALMLEAFEQLLGKRRELLGE
Class prediction: non-AMP
Probability of being AMP:  [0.47721216]
Input peptide sequence: SFDRVMNAIEEN
Class prediction: non-AMP
Probability of being AMP:  [0.47143528]
Input peptide sequence: SVADHSYRVAFITLLLAEELKKK
Class prediction: non-AMP
Probability of being AMP:  [0.48379597]
Input peptide sequence: VMPSLLQAFDLTA
Class prediction: non-AMP
Probability of being AMP:  [0.47037894]
Input peptide sequence: KEEGYFYFRSRLSQ
Class prediction: non-AMP
Probability of being AMP:  [0.4718237]
Input peptide sequence: MRKWFHNVLSSGQLLADKWP